In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
import django
from tqdm.auto import tqdm

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

sys.path.append("..")
os.environ['DJANGO_SETTINGS_MODULE'] = 'news_labelling.settings'
django.setup()

from api.models import Comment, ArticleLabel, CommentLabel, Article, Batch
from django.contrib.auth.models import User

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
skipped_labels = ArticleLabel.objects.filter(is_interesting=False).prefetch_related(
        'article', 'user'
)

annotated = set()
skipped = set()

skipped_articles = {label.article for label in skipped_labels}
comments_to_reannotate = 0
for article in skipped_articles:
    skipped.add(article)
    if article.labels.filter(is_interesting=True).exists():
        article_labels = article.labels.filter(is_interesting=True).prefetch_related('comment_labels', 'comment_labels__comment')
        
        to_reannotate = {comment_label.comment
            for article_label in article_labels
            for comment_label in article_label.comment_labels.all()
            if comment_label.is_hateful
        }
        
        comments_to_reannotate += len(to_reannotate)
        
        annotated.add(article)


print(f"De los {len(skipped)} artículos salteados algún otro anotador anotó {len(annotated)}")
print(f"Habría que reanotar {comments_to_reannotate} comentarios")

De los 96 artículos salteados algún otro anotador anotó 74
Habría que reanotar 187 comentarios


## 3ra anotación

¿Cuántos tendríamos que meterle 3ra anotación?

Segurísimo: los que tienen al menos una anotación positiva

In [11]:
possibly_hateful = {label.comment for label in CommentLabel.objects.filter(is_hateful=True)}

print(len(possibly_hateful))

3777


Esto ahora mismo serían aproximadamente 75 artículos de 50 comentarios

¿Cuántos comentarios anotados tenemos?

In [16]:
len({label.comment for label in CommentLabel.objects.all()})

18295

In [15]:
3777/18295

0.20644984968570648

Necesitamos reanotar cerca del 20% de los comentarios. O sea, un 20% más del trabajo ya hecho